# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [8]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
INPUT_DIR = "/scratch/back_up/reward_competition_extention/data"
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs
TONE_DIN = "dio_ECU_Din1"
TONE_STATE = 1
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [9]:
COLS_TO_KEEP = ['session_dir', 'recording', 'metadata_dir', 'metadata_file',
'original_file', 'filename', 'session_path', 'all_subjects',
       'current_subject', 'event_timestamps', 'video_name',
       'video_timestamps', 'event_frames', 'first_item_data']

In [10]:
# TODO: Find way not to hard code this
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/standard/2023_06_*/*.rec")


In [11]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/standard/2023_06_14/20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/standard/2023_06_13/20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.rec',
 '/scratch/back_up/reward_competition_extention/data/standard/2023_06_12/20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.rec',
 '/scratch/back_up/reward_competition_extention/data/standard/2023_06_12/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/standard/2023_06_16/20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

In [12]:
def recursive_dictionary():
    return defaultdict(recursive_dictionary)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [13]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# LOOP 1: Extracting all the Trodes

- Getting all the data from all the exported Trodes files

- Getting all the data from all the exported Trodes files and saving it to `session_to_trodes_data`
    - Creates a dictionary with the structure of:
        - `{dir_name: {file_name: metadata, file_name_2: metadata_2}, dir_name_2: {file_name_3: metadata_3, file_name_4: metadata_4}}`

In [14]:
# Saving the trodes data for each session
# Each key is a session name
# Each value is a dictionary of every recording file in that session
session_to_trodes_data = recursive_dictionary()


# Saving the path of the session recording
session_to_path = {}

# Going through each session recording
# Which includes all the recordings from all the miniloggers and cameras
for session_path in ALL_SESSION_DIR:   
    try:
        # Getting the name of the session from the path
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        # Reading the trodes data for every recording file in the session directory
        session_to_trodes_data[session_basename] = trodes.read_exported.organize_all_trodes_export(session_path)
        
        session_to_path[session_basename] = session_path
    except Exception as e: 
        print(e)


Current Session: 20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2


/nancy/projects/reward_competition_extention/src/trodes/read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


Skipping file 20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230614_114041_standard_comp_to_training_D3_subj_1-1_t1b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Current Session: 20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4
Skipping file 20230613_105657_standard_comp_to_training_D2_subj_1-4_t4b3L_box2_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230613_105657_standard_comp_to_training_D2_subj_1-1_t1b2L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Current Session: 20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3
Skipping file 20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.timest

In [15]:
session_to_trodes_data

defaultdict(<function __main__.recursive_dictionary()>,
            {'20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2': defaultdict(dict,
                         {'20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged': {'DIO': {'dio_ECU_Dout4': {'description': 'State change data for one digital channel. Display_order is 1-based',
                             'byte_order': 'little endian',
                             'original_file': '20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged.rec',
                             'clockrate': '20000',
                             'trodes_version': '2.4.0',
                             'compile_date': 'May 24 2023',
                             'compile_time': '10:11:04',
                             'qt_version': '6.2.2',
                             'commit_tag': 'heads/Release_2.4.0-0-g1eecf3b7',
                             'controller_firmware': '3.17',
                             'heads

- Adding the video timestamps

In [16]:
for session_path in ALL_SESSION_DIR:   
    try:
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        file_to_video_timestamps = {}
        for video_timestamps in glob.glob(os.path.join(session_path, "*cameraHWSync")):
            video_basename = os.path.basename(video_timestamps)
            print("Current Video Name: {}".format(video_basename))
            timestamp_array = trodes.read_exported.read_trodes_extracted_data_file(video_timestamps)
            if "video_timestamps" not in session_to_trodes_data[session_basename][session_basename]:
                session_to_trodes_data[session_basename][session_basename]["video_timestamps"] = defaultdict(dict)
            session_to_trodes_data[session_basename][session_basename]["video_timestamps"][video_basename.split(".")[-3]] = timestamp_array
    
    
    except Exception as e: 
        print(e)

Current Session: 20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2
Current Video Name: 20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.3.videoTimeStamps.cameraHWSync
Current Video Name: 20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.2.videoTimeStamps.cameraHWSync
Current Video Name: 20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1.videoTimeStamps.cameraHWSync
Current Session: 20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4
Current Video Name: 20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1.videoTimeStamps.cameraHWSync
Current Video Name: 20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.2.videoTimeStamps.cameraHWSync
Current Session: 20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3
Current Video Name: 20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.2.videoTimeStamps.cameraHWSync
Current Video Name: 20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.1.v

In [17]:
session_to_trodes_data[session_basename][session_basename]["video_timestamps"]

defaultdict(dict,
            {'1': {'clock rate': '30000',
              'fields': '<PosTimestamp uint32><HWframeCount uint32><HWTimestamp uint64>',
              'data': array([(  309047, 0, 0), (  309047, 0, 0), (  310433, 0, 0), ...,
                     (67787037, 0, 0), (67787037, 0, 0), (67788422, 0, 0)],
                    dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')]),
              'filename': '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1.videoTimeStamps.cameraHWSync'},
             '2': {'clock rate': '30000',
              'fields': '<PosTimestamp uint32><HWframeCount uint32><HWTimestamp uint64>',
              'data': array([(  309047, 0, 0), (  309047, 0, 0), (  310433, 0, 0), ...,
                     (67785651, 0, 0), (67787037, 0, 0), (67787890, 0, 0)],
                    dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')]),
              'filename': '20230616_111904_standard_comp_to_t

- Creating a dataframe the dictionary with a column for:
  - Session directory
  - Recording name
  - Metadata directory
  - Metadata file
  - And a column for each metadata

In [18]:
# Creating a dataframe from the nested dictionary
trodes_metadata_df = pd.DataFrame.from_dict({(i,j,k,l): session_to_trodes_data[i][j][k][l] 
                           for i in session_to_trodes_data.keys() 
                           for j in session_to_trodes_data[i].keys()
                           for k in session_to_trodes_data[i][j].keys()
                           for l in session_to_trodes_data[i][j][k].keys()},
                           orient='index')

# Resetting the index and renaming the columns
trodes_metadata_df = trodes_metadata_df.reset_index()
trodes_metadata_df = trodes_metadata_df.rename(columns={'level_0': 'session_dir', 'level_1': 'recording', 'level_2': 'metadata_dir', 'level_3': 'metadata_file'}, errors="ignore")

# Adding the session path to the dataframe
trodes_metadata_df["session_path"] = trodes_metadata_df["session_dir"].map(session_to_path)

In [19]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_timestamp,direction,id,display_order,fields,data,filename,decimation,clock rate,session_path
0,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Dout4,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,34373723,output,ECU_Dout4,5,<time uint32><state uint8>,"[[34373723, 0]]",20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...
1,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,34373723,input,ECU_Din2,6,<time uint32><state uint8>,"[[34373723, 0], [34395880, 1], [34401285, 0], ...",20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...
2,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Dout2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,34373723,output,ECU_Dout2,3,<time uint32><state uint8>,"[[34373723, 0]]",20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...
3,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,34373723,input,ECU_Din4,9,<time uint32><state uint8>,"[[34373723, 0]]",20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...
4,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,34373723,output,ECU_Dout1,2,<time uint32><state uint8>,"[[34373723, 0]]",20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...


In [20]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_timestamp,direction,id,display_order,fields,data,filename,decimation,clock rate,session_path
126,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[8798884, 0, 0], [8800270, 0, 0], [8801656, 0...",20230612_101430_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...
127,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[7978450, 0, 0], [7979510, 0, 0], [7979835, 0...",20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...
128,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[7978450, 0, 0], [7979597, 0, 0], [7979835, 0...",20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...
129,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[309047, 0, 0], [309047, 0, 0], [310433, 0, 0...",20230616_111904_standard_comp_to_training_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...
130,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[309047, 0, 0], [309047, 0, 0], [310433, 0, 0...",20230616_111904_standard_comp_to_training_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...


- Getting the first item from each tuple in the arrays in the `data` column
  - This first item is usually just the timestamp

In [21]:
trodes_metadata_df["data"].iloc[0]

array([(34373723, 0)], dtype=[('time', '<u4'), ('state', 'u1')])

In [22]:
# Getting the dtype name of each column in the numpy array
trodes_metadata_df["first_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[0])
# Getting the first item of each column in the numpy array
trodes_metadata_df["first_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[0]])


In [23]:
# Same as above but for the last column
trodes_metadata_df["last_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[-1])
trodes_metadata_df["last_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[-1]])

In [24]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,fields,data,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
0,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Dout4,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[34373723, 0]]",20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[34373723],state,[0]
1,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[34373723, 0], [34395880, 1], [34401285, 0], ...",20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 34395880, 34401285, 34485478, 34500...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ..."
2,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Dout2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[34373723, 0]]",20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[34373723],state,[0]
3,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[34373723, 0]]",20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[34373723],state,[0]
4,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[34373723, 0]]",20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[34373723],state,[0]


In [25]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,fields,data,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
126,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[8798884, 0, 0], [8800270, 0, 0], [8801656, 0...",20230612_101430_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[8798884, 8800270, 8801656, 8803042, 8803042, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
127,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[7978450, 0, 0], [7979510, 0, 0], [7979835, 0...",20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[7978450, 7979510, 7979835, 7981221, 7982607, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
128,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[7978450, 0, 0], [7979597, 0, 0], [7979835, 0...",20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[7978450, 7979597, 7979835, 7981221, 7982607, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
129,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[309047, 0, 0], [309047, 0, 0], [310433, 0, 0...",20230616_111904_standard_comp_to_training_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[309047, 309047, 310433, 311819, 313205, 31320...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
130,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[309047, 0, 0], [309047, 0, 0], [310433, 0, 0...",20230616_111904_standard_comp_to_training_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[309047, 309047, 310433, 310433, 311819, 31320...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [26]:
trodes_metadata_df["recording"].unique()

array(['20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_t1b3L_box1_merged',
       '20230613_105657_standard_comp_to_training_D2_subj_1-4_t4b3L_box2_merged',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_t1b2L_box1_merged',
       '20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged',
       '20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged',
       '20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged',
       '20230616_111904_standard_comp_to_training_D4_subj_1-2_t2b2L_box2_merged',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4',
       '20230612_101430_standard_com

## Getting the subject information from the metadata

In [27]:
trodes_metadata_df["all_subjects"] = trodes_metadata_df["recording"].apply(lambda x: x.replace("-","_").split("subj")[-1].split("t")[0].strip("_").replace("_",".").split(".and."))
trodes_metadata_df["all_subjects"] = trodes_metadata_df["all_subjects"].apply(lambda x: sorted([i.strip().strip(".") for i in x]))

In [28]:
trodes_metadata_df["current_subject"] = trodes_metadata_df["all_subjects"].apply(lambda x: x[0] if len(x) == 1 else np.nan)

In [29]:
trodes_metadata_df["all_subjects"]

0           [1.2]
1           [1.2]
2           [1.2]
3           [1.2]
4           [1.2]
          ...    
126    [1.3, 1.4]
127    [1.1, 1.2]
128    [1.1, 1.2]
129    [1.2, 1.4]
130    [1.2, 1.4]
Name: all_subjects, Length: 131, dtype: object

In [30]:
trodes_metadata_df["current_subject"]

0      1.2
1      1.2
2      1.2
3      1.2
4      1.2
      ... 
126    NaN
127    NaN
128    NaN
129    NaN
130    NaN
Name: current_subject, Length: 131, dtype: object

In [31]:
ALL_SUBJECTS_COL = "all_subjects"

## Dropping all the rows with unneeded metadata

In [32]:
trodes_metadata_df["metadata_dir"].unique()

array(['DIO', 'analog', 'time', 'raw', 'video_timestamps'], dtype=object)

In [33]:
METADATA_TO_KEEP = ['raw', 'DIO', 'video_timestamps']

In [34]:
trodes_metadata_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(METADATA_TO_KEEP)]

In [35]:
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("out")]
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("coordinates")]


In [36]:
trodes_metadata_df = trodes_metadata_df.reset_index(drop=True)

# Getting the first time stamp of each recording

In [37]:
trodes_raw_df = trodes_metadata_df[(trodes_metadata_df["metadata_dir"] == "raw") & (trodes_metadata_df["metadata_file"] == "timestamps")].copy()


In [38]:
trodes_raw_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
4,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,raw,timestamps,Raw timestamps,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 34373724, 34373725, 34373726, 34373...",time,"[34373723, 34373724, 34373725, 34373726, 34373...",[1.2],1.2
9,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,raw,timestamps,Raw timestamps,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 34373724, 34373725, 34373726, 34373...",time,"[34373723, 34373724, 34373725, 34373726, 34373...",[1.1],1.1
10,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,raw,timestamps,Raw timestamps,little endian,20230613_105657_standard_comp_to_training_D2_s...,20000,2.4.0,May 24 2023,...,20230613_105657_standard_comp_to_training_D2_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[7454593, 7454594, 7454595, 7454596, 7454597, ...",time,"[7454593, 7454594, 7454595, 7454596, 7454597, ...",[1.4],1.4
15,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,raw,timestamps,Raw timestamps,little endian,20230613_105657_standard_comp_to_training_D2_s...,20000,2.4.0,May 24 2023,...,20230613_105657_standard_comp_to_training_D2_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[7454593, 7454594, 7454595, 7454596, 7454597, ...",time,"[7454593, 7454594, 7454595, 7454596, 7454597, ...",[1.1],1.1
24,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,raw,timestamps,Raw timestamps,little endian,20230612_101430_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_101430_standard_comp_to_training_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[8798886, 8798887, 8798888, 8798889, 8798890, ...",time,"[8798886, 8798887, 8798888, 8798889, 8798890, ...",[1.3],1.3
25,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,raw,timestamps,Raw timestamps,little endian,20230612_101430_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_101430_standard_comp_to_training_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[8798886, 8798887, 8798888, 8798889, 8798890, ...",time,"[8798886, 8798887, 8798888, 8798889, 8798890, ...",[1.4],1.4
34,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,raw,timestamps,Raw timestamps,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",[1.2],1.2
35,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,raw,timestamps,Raw timestamps,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",[1.1],1.1
40,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_co

In [39]:
trodes_raw_df["first_timestamp"] = trodes_raw_df["first_item_data"].apply(lambda x: x[0])

In [40]:
trodes_raw_df["recording"].iloc[0]

'20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged'

In [41]:
recording_to_first_timestamp = trodes_raw_df.set_index('session_dir')['first_timestamp'].to_dict()

In [42]:
recording_to_first_timestamp

{'20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2': 34373723,
 '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4': 7454593,
 '20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3': 8798886,
 '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1': 7977066,
 '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2': 307664}

#TODO: Make sure that the first timestamp is the same for all the recordings in the session

In [43]:
trodes_metadata_df["first_timestamp"] = trodes_metadata_df["session_dir"].map(recording_to_first_timestamp)

In [44]:
trodes_metadata_df["first_timestamp"]

0     34373723
1     34373723
2     34373723
3     34373723
4     34373723
        ...   
56     8798886
57     7977066
58     7977066
59      307664
60      307664
Name: first_timestamp, Length: 61, dtype: int64

# Getting the event timestamps

In [45]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 34395880, 34401285, 34485478, 34500...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
1,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[34373723],state,[0],[1.2],1.2
2,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 67422061, 67433461, 67440261, 67441...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
3,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 35230678, 35430680, 37630681, 37830...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
4,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,raw,timestamps,Raw timestamps,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 34373724, 34373725, 34373726, 34373...",time,"[34373723, 34373724, 34373725, 34373726, 34373...",[1.2],1.2


In [46]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
56,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230612_101430_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[8798884, 8800270, 8801656, 8803042, 8803042, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.3, 1.4]",NaN
57,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[7978450, 7979510, 7979835, 7981221, 7982607, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",NaN
58,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[7978450, 7979597, 7979835, 7981221, 7982607, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",NaN
59,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[309047, 309047, 310433, 311819, 313205, 31320...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.2, 1.4]",NaN
60,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[309047, 309047, 310433, 310433, 311819, 31320...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.2, 1.4]",NaN


In [47]:
# trodes_state_df = trodes_metadata_df[trodes_metadata_df["last_dtype_name"] == "state"].copy()

# Filtering for digital IO channels
trodes_state_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(["DIO"])].copy()
# Filtering for tone and port entry related channels
trodes_state_df = trodes_metadata_df[trodes_metadata_df["id"].isin(["ECU_Din1", "ECU_Din2"])].copy()


In [48]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 34395880, 34401285, 34485478, 34500...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
3,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 35230678, 35430680, 37630681, 37830...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
7,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 34395880, 34401285, 34485478, 34500...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1
8,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 35230678, 35430680, 37630681, 37830...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1
12,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230613_105657_standard_comp_to_training_D2_s...,20000,2.4.0,May 24 2023,...,20230613_105657_standard_comp_to_training_D2_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[7454593, 7503951, 7522151, 7524748, 7533948, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
14,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230613_105657_standard_comp_to_training_D2_s...,20000,2.4.0,May 24 2023,...,20230613_105657_standard_comp_to_training_D2_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[7454593, 8373348, 8573351, 10773348, 10973350...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4
17,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230613_105657_standard_comp_to_training_D2_s...,20000,2.4.0,May 24 2023,...,20230613_105657_standard_comp_to_training_D2_s...,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[7454593, 7503951, 7522151, 7524748, 7533948, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1
18,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230613_105657_standard_comp_to_training_D2_s...,20000,2.4.0,May 24 2023,...,20230613_10

In [49]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: np.column_stack([np.where(x["last_item_data"] == 1)[0], np.where(x["last_item_data"] == 1)[0]+1]), axis=1)

In [50]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: x["event_indexes"][x["event_indexes"][:, 1] <= x["first_item_data"].shape[0] - 1], axis=1)

In [51]:
trodes_state_df["event_timestamps"] = trodes_state_df.apply(lambda x: x["first_item_data"][x["event_indexes"]], axis=1)

In [52]:
trodes_state_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps
0,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 34395880, 34401285, 34485478, 34500...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[34395880, 34401285], [34485478, 34500880], [..."
3,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 35230678, 35430680, 37630681, 37830...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[35230678, 35430680], [37630681, 37830676], [..."
7,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 34395880, 34401285, 34485478, 34500...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[34395880, 34401285], [34485478, 34500880], [..."
8,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[34373723, 35230678, 35430680, 37630681, 37830...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[35230678, 35430680], [37630681, 37830676], [..."
12,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230613_105657_standard_comp_to_training_D2_s...,20000,2.4.0,May 24 2023,...,NaN,/scratch/back_up/reward_competition_extention/...,time,"[7454593, 7503951, 7522151, 7524748, 7533948, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7503951, 7522151], [7524748, 7533948], [7815..."


## Updating the video timestamps

## Syncing up the video frame data

In [53]:
# Getting the rows that are the metadata for the video timestamps
trodes_video_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"] == "video_timestamps"].copy().reset_index(drop=True)

# Filtering for the first video only
# This only applies to this pilot data where we are only looking the at competition data
trodes_video_df = trodes_video_df[trodes_video_df["metadata_file"] == "1"].copy()

In [54]:
trodes_video_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
2,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230614_114041_standard_comp_to_training_D3_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[34373721, 34375107, 34376493, 34377879, 34377...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",NaN
3,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230613_105657_standard_comp_to_training_D2_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[7455977, 7455977, 7457363, 7458749, 7458749, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.4]",NaN
6,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230612_101430_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[8798884, 8800270, 8801656, 8803042, 8803042, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.3, 1.4]",NaN
7,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[7978450, 7979510, 7979835, 7981221, 7982607, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",NaN
9,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230616_111904_standard_comp_to_training_D4_s...,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[309047, 309047, 310433, 311819, 313205, 31320...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.2, 1.4]",NaN


In [55]:
# Making the video timestamps into an evenly distributed array
trodes_video_df["video_timestamps"] = trodes_video_df["first_item_data"]

In [56]:
# Removing the columns that are no longer needed
trodes_video_df = trodes_video_df[["filename", "video_timestamps", "session_dir"]].copy()

In [57]:
# Renaming the filename so that we can merge with other dataframes with the same column name
trodes_video_df = trodes_video_df.rename(columns={"filename": "video_name"})

In [58]:
trodes_video_df

,video_name,video_timestamps,session_dir
2,20230614_114041_standard_comp_to_training_D3_s...,"[34373721, 34375107, 34376493, 34377879, 34377...",20230614_114041_standard_comp_to_training_D3_s...
3,20230613_105657_standard_comp_to_training_D2_s...,"[7455977, 7455977, 7457363, 7458749, 7458749, ...",20230613_105657_standard_comp_to_training_D2_s...
6,20230612_101430_standard_comp_to_training_D1_s...,"[8798884, 8800270, 8801656, 8803042, 8803042, ...",20230612_101430_standard_comp_to_training_D1_s...
7,20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...",20230612_112630_standard_comp_to_training_D1_s...
9,20230616_111904_standard_comp_to_training_D4_s...,"[309047, 309047, 310433, 311819, 313205, 31320...",20230616_111904_standard_comp_to_training_D4_s...


- Adding each video as a row to each state row

In [59]:
trodes_state_df = pd.merge(trodes_state_df, trodes_video_df, on=["session_dir"], how="left")

In [60]:
trodes_state_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps,video_name,video_timestamps
0,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,time,"[34373723, 34395880, 34401285, 34485478, 34500...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[34395880, 34401285], [34485478, 34500880], [...",20230614_114041_standard_comp_to_training_D3_s...,"[34373721, 34375107, 34376493, 34377879, 34377..."
1,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,time,"[34373723, 35230678, 35430680, 37630681, 37830...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[35230678, 35430680], [37630681, 37830676], [...",20230614_114041_standard_comp_to_training_D3_s...,"[34373721, 34375107, 34376493, 34377879, 34377..."
2,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,time,"[34373723, 34395880, 34401285, 34485478, 34500...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[34395880, 34401285], [34485478, 34500880], [...",20230614_114041_standard_comp_to_training_D3_s...,"[34373721, 34375107, 34376493, 34377879, 34377..."
3,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,time,"[34373723, 35230678, 35430680, 37630681, 37830...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[35230678, 35430680], [37630681, 37830676], [...",20230614_114041_standard_comp_to_training_D3_s...,"[34373721, 34375107, 34376493, 34377879, 34377..."
4,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230613_105657_standard_comp_to_training_D2_s...,20000,2.4.0,May 24 2023,...,time,"[7454593, 7503951, 7522151, 7524748, 7533948, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7503951, 7522151], [7524748, 7533948], [7815...",20230613_105657_standard_comp_to_training_D2_s...,"[7455977, 7455977, 7457363, 7458749, 7458749, ..."


## Finding the closest frame to each event

In [61]:
def find_closest_indices(array1, array2):
    array1_flat = array1.flatten()
    indices = np.array([np.abs(array2 - num).argmin() for num in array1_flat])
    return indices.reshape(array1.shape)

In [62]:
trodes_state_df["event_timestamps"].iloc[1]

array([[ 35230678,  35430680],
       [ 37630681,  37830676],
       [ 39930677,  40130675],
       [ 41730676,  41930676],
       [ 42830676,  43030676],
       [ 43830675,  44030680],
       [ 45730674,  45930671],
       [ 47230673,  47430673],
       [ 49330670,  49530672],
       [ 50330672,  50530671],
       [ 51930671,  52130668],
       [ 53230668,  53430668],
       [ 54530669,  54730667],
       [ 55530669,  55730672],
       [ 56930669,  57130666],
       [ 58830665,  59030672],
       [ 60430664,  60630664],
       [ 61830667,  62030666],
       [ 64030663,  64230662],
       [ 65230662,  65430662],
       [ 71230662,  71430662],
       [ 72430662,  72630659],
       [ 73530661,  73730662],
       [ 74830661,  75030661],
       [ 75930660,  76130658],
       [ 76930658,  77130660],
       [ 77930657,  78130657],
       [ 80030659,  80230656],
       [ 82430658,  82630668],
       [ 84830657,  85030657],
       [ 87030653,  87230654],
       [ 88730655,  88930652],
       [

In [63]:
trodes_state_df["event_frames"] = trodes_state_df.apply(lambda x: find_closest_indices(x["event_timestamps"], x["video_timestamps"]), axis=1)

In [64]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps,video_name,video_timestamps,event_frames
0,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,"[34373723, 34395880, 34401285, 34485478, 34500...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[34395880, 34401285], [34485478, 34500880], [...",20230614_114041_standard_comp_to_training_D3_s...,"[34373721, 34375107, 34376493, 34377879, 34377...","[[21, 27], [111, 126], [278, 321], [339, 393],..."
1,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,"[34373723, 35230678, 35430680, 37630681, 37830...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[35230678, 35430680], [37630681, 37830676], [...",20230614_114041_standard_comp_to_training_D3_s...,"[34373721, 34375107, 34376493, 34377879, 34377...","[[855, 1055], [3251, 3450], [5546, 5747], [734..."
2,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,"[34373723, 34395880, 34401285, 34485478, 34500...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[34395880, 34401285], [34485478, 34500880], [...",20230614_114041_standard_comp_to_training_D3_s...,"[34373721, 34375107, 34376493, 34377879, 34377...","[[21, 27], [111, 126], [278, 321], [339, 393],..."
3,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230614_114041_standard_comp_to_training_D3_s...,20000,2.4.0,May 24 2023,...,"[34373723, 35230678, 35430680, 37630681, 37830...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[35230678, 35430680], [37630681, 37830676], [...",20230614_114041_standard_comp_to_training_D3_s...,"[34373721, 34375107, 34376493, 34377879, 34377...","[[855, 1055], [3251, 3450], [5546, 5747], [734..."
4,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230613_105657_standard_comp_to_training_D2_s...,20000,2.4.0,May 24 2023,...,"[7454593, 7503951, 7522151, 7524748, 7533948, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7503951, 7522151], [7524748, 7533948], [7815...",20230613_105657_standard_comp_to_training_D2_s...,"[7455977, 7455977, 7457363, 7458749, 7458749, ...","[[49, 67], [70, 79], [360, 366], [460, 469], [..."
5,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230613_105657_standard_comp_to_training_D2_s...,20000,2.4.0,May 24 2023,...,"[7454593, 8373348, 8573351, 10773348, 10973350...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.4],1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[8373348, 8573351], [10773348, 10973350], [13...",20230613_105657_standard_comp_to_training_D

## Combine raw and state dataframes

In [65]:
RAW_COLS_TO_KEEP = ['session_dir',
 'recording',
 'original_file',
 'filename',
 'session_path',
 'current_subject',
 'first_item_data']

In [66]:
STATE_COLS_TO_KEEP = ['session_dir',
 'metadata_file',
 'event_timestamps',
 'video_name',
 'video_timestamps',
 'event_frames',]

In [67]:
trodes_state_df = trodes_state_df[STATE_COLS_TO_KEEP].drop_duplicates(subset=["session_dir", "metadata_file"]).sort_values(["session_dir", "metadata_file"]).reset_index(drop=True).copy()

In [68]:
same_columns = ['session_dir', 'video_name']
different_columns = ['metadata_file', 'event_frames', 'event_timestamps']

In [69]:
trodes_state_df = trodes_state_df.groupby(same_columns).agg({**{col: 'first' for col in trodes_state_df.columns if col not in same_columns + different_columns}, **{col: lambda x: x.tolist() for col in different_columns}}).reset_index()

In [70]:
trodes_state_df["tone_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[0])
trodes_state_df["port_entry_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[1])

trodes_state_df["tone_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[0])
trodes_state_df["port_entry_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[1])


In [71]:
trodes_state_df = trodes_state_df.drop(columns=["event_timestamps", "event_frames"], errors="ignore")

In [72]:
trodes_state_df

,session_dir,video_name,video_timestamps,metadata_file,tone_timestamps,port_entry_timestamps,tone_frames,port_entry_frames
0,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[8798884, 8800270, 8801656, 8803042, 8803042, ...","[dio_ECU_Din1, dio_ECU_Din2]","[[9781115, 9981112], [12181113, 12381110], [14...","[[9289915, 9314113], [9318312, 9357515], [9358...","[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640..."
1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...","[dio_ECU_Din1, dio_ECU_Din2]","[[9103808, 9303807], [11503806, 11703806], [13...","[[8169811, 8226416], [8366813, 8384208], [8894...","[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948..."
2,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[7455977, 7455977, 7457363, 7458749, 7458749, ...","[dio_ECU_Din1, dio_ECU_Din2]","[[8373348, 8573351], [10773348, 10973350], [13...","[[7503951, 7522151], [7524748, 7533948], [7815...","[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [..."
3,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,"[34373721, 34375107, 34376493, 34377879, 34377...","[dio_ECU_Din1, dio_ECU_Din2]","[[35230678, 35430680], [37630681, 37830676], [...","[[34395880, 34401285], [34485478, 34500880], [...","[[855, 1055], [3251, 3450], [5546, 5747], [734...","[[21, 27], [111, 126], [278, 321], [339, 393],..."
4,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,"[309047, 309047, 310433, 311819, 313205, 31320...","[dio_ECU_Din1, dio_ECU_Din2]","[[395679, 595679], [2795675, 2995676], [509567...","[[646678, 647879], [648676, 693876], [694279, ...","[[87, 287], [2483, 2683], [4778, 4979], [6575,...","[[338, 339], [341, 385], [385, 482], [696, 703..."


In [73]:
trodes_raw_df = trodes_raw_df[RAW_COLS_TO_KEEP].reset_index(drop=True).copy()

In [74]:
trodes_raw_df

,session_dir,recording,original_file,filename,session_path,current_subject,first_item_data
0,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,/scratch/back_up/reward_competition_extention/...,1.2,"[34373723, 34373724, 34373725, 34373726, 34373..."
1,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,/scratch/back_up/reward_competition_extention/...,1.1,"[34373723, 34373724, 34373725, 34373726, 34373..."
2,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,1.4,"[7454593, 7454594, 7454595, 7454596, 7454597, ..."
3,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,1.1,"[7454593, 7454594, 7454595, 7454596, 7454597, ..."
4,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,1.3,"[8798886, 8798887, 8798888, 8798889, 8798890, ..."
5,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,1.4,"[8798886, 8798887, 8798888, 8798889, 8798890, ..."
6,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,1.2,"[7977066, 7977067, 7977068, 7977069, 7977070, ..."
7,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,1.1,"[7977066, 7977067, 7977068, 7977069, 7977070, ..."
8,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,1.4,"[307664, 307665, 307666, 307667, 307668, 30766..."
9,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,1.2,"[307664, 307665, 307666, 307667, 307668, 30766..."


In [75]:
trodes_final_df = pd.merge(trodes_raw_df, trodes_state_df, on=["session_dir"], how="left")

In [83]:
trodes_final_df = trodes_final_df.rename(columns={"first_item_data": "raw_timestamps"})
trodes_final_df = trodes_final_df.drop(columns=["metadata_file"], errors="ignore")
trodes_final_df = trodes_final_df.sort_values(["session_dir", "recording"]).reset_index(drop=True).copy()

In [84]:
trodes_final_df

,session_dir,recording,original_file,filename,session_path,current_subject,raw_timestamps,video_name,video_timestamps,tone_timestamps,port_entry_timestamps,tone_frames,port_entry_frames
0,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,1.3,"[8798886, 8798887, 8798888, 8798889, 8798890, ...",20230612_101430_standard_comp_to_training_D1_s...,"[8798884, 8800270, 8801656, 8803042, 8803042, ...","[[9781115, 9981112], [12181113, 12381110], [14...","[[9289915, 9314113], [9318312, 9357515], [9358...","[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640..."
1,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,1.4,"[8798886, 8798887, 8798888, 8798889, 8798890, ...",20230612_101430_standard_comp_to_training_D1_s...,"[8798884, 8800270, 8801656, 8803042, 8803042, ...","[[9781115, 9981112], [12181113, 12381110], [14...","[[9289915, 9314113], [9318312, 9357515], [9358...","[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640..."
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,1.1,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...","[[9103808, 9303807], [11503806, 11703806], [13...","[[8169811, 8226416], [8366813, 8384208], [8894...","[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948..."
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,1.2,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...","[[9103808, 9303807], [11503806, 11703806], [13...","[[8169811, 8226416], [8366813, 8384208], [8894...","[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948..."
4,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,1.1,"[7454593, 7454594, 7454595, 7454596, 7454597, ...",20230613_105657_standard_comp_to_training_D2_s...,"[7455977, 7455977, 7457363, 7458749, 7458749, ...","[[8373348, 8573351], [10773348, 10973350], [13...","[[7503951, 7522151], [7524748, 7533948], [7815...","[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [..."
5,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,1.4,"[7454593, 7454594, 7454595, 7454596, 7454597, ...",20230613_105657_standard_comp_to_training_D2_s...,"[7455977, 7455977, 7457363, 7458749, 7458749, ...","[[8373348, 8573351], [10773348, 10973350], [13...","[[7503951, 7522151], [7524748, 7533948], [7815...","[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [..."
6,20230614_114041_standard_comp_to_training_D3_s...,20230614_114041_standard_comp_to_training_D3_s...,2023

In [85]:
trodes_final_df.to_pickle(os.path.join(OUTPUT_DIR, "{}_trodes_metadata.pkl".format(OUTPUT_PREFIX)))